In [52]:
import pandas as pd
from Main import settings, features, pf_set
import time
import pickle
from pandas.tseries.offsets import DateOffset
import Prepare_Data
import data_run_files
import return_prediction_functions
from return_prediction_functions import rff
import numpy as np
from pandas.tseries.offsets import MonthEnd
from sklearn.linear_model import Ridge
import General_Functions
import pickle
import Estimate_Covariance_Matrix
import os
import prepare_portfolio_data
import portfolio_choice_functions

start_date = pd.to_datetime('1952-12-31')

#Relevante paths
output_path = "./data_fifty/"
pickle_file_path = "./data_fifty/model_1.pkl"
wealth_path = "./data_fifty/wealth.csv"
barra_file_path = "./data_fifty/barra_cov.pkl"
output_path_market_returns = "data_fifty/market_returns_test.csv"

risk_free_path = "./data_fifty/risk_free_test.csv" 
output_data_ret_csv = "./data_fifty/data_ret.csv"  
output_data_ret_ld1_csv = "./data_fifty/data_ret_ld1.csv"

file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"

risk_free = data_run_files.process_risk_free_rate(risk_free_path, start_date)
daily_path = "./data_fifty/daily.csv"
chars_path = "./data_fifty/chars_behandlet.parquet"


Filtrering udført. Antal rækker efter filtrering: 841
Fil gemt som ./data_test/risk_free_test.csv


# Indlæser wealth, chars, daily, data_ret og data_ret_ld1

In [53]:
wealth = pd.read_csv(wealth_path)
wealth['eom'] = pd.to_datetime(wealth['eom'])
data_ret = pd.read_csv(output_data_ret_csv)
data_ret_ld1 = pd.read_csv(output_data_ret_ld1_csv)
data_ret['eom'] = pd.to_datetime(data_ret['eom'])
data_ret_ld1['eom'] = pd.to_datetime(data_ret_ld1['eom'])
data_ret_ld1['eom_ret'] = pd.to_datetime(data_ret_ld1['eom_ret'])
daily = pd.read_csv(daily_path, parse_dates=["date", "eom"])
chars = pd.read_parquet(chars_path)

Disse er checket og se ser korrekte ud wuhu :D

In [54]:
data_ret

,id,eom,ret_ld1,ret_ld2,ret_ld3,ret_ld4,ret_ld5,ret_ld6,ret_ld7,ret_ld8,ret_ld9,ret_ld10,ret_ld11,ret_ld12
0,10104,1986-04-30,-0.064934,0.031988,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897
1,10104,1986-05-31,0.031988,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814
2,10104,1986-06-30,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517
3,10104,1986-07-31,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517,0.018414
4,10104,1986-08-31,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517,0.018414,0.171080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90698,92655,2022-07-31,-0.044328,-0.026268,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90699,92655,2022-08-31,-0.026268,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90700,92655,2022-09-30,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90701,92655,2022-10-31,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Hent Predictions :D

### Indlæsning af Pickle fil (predictions)

In [55]:
with open(pickle_file_path, "rb") as file:
    model_1 = pickle.load(file)

#print(model_1) #Vi bruger dog ikke ikke denne i selve 

# Barra COV

Just a quick check

In [56]:
with open(barra_file_path, "rb") as file:
    barra_cov = pickle.load(file)

In [57]:
print(barra_cov['2022-11-30'])

{'fct_load':        BusEq  Chems  Durbl  Enrgy  Hlth  Manuf  Money  NoDur  Other  Shops  \
id                                                                           
10104    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10107    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10138    0.0    0.0    0.0    0.0   0.0    0.0    1.0    0.0    0.0    0.0   
10696    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
11308    0.0    0.0    0.0    0.0   0.0    0.0    0.0    1.0    0.0    0.0   
...      ...    ...    ...    ...   ...    ...    ...    ...    ...    ...   
87299    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
87432    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
87447    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
87717    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
92655    0.0    0.0    0.0    0.0   0.0    0.0    1

In [58]:
print(barra_cov['2022-11-30']['ivol_vec'])

id
10104    0.004043
10107    0.002393
10138    0.002996
10696    0.003696
11308    0.001280
           ...   
87299    0.004106
87432    0.003534
87447    0.003139
87717    0.003166
92655    0.002555
Name: res_vol, Length: 180, dtype: float64


# Prepare Portfolio Data - tilføj return predictions, lambda list og datoer

In [59]:
chars = prepare_portfolio_data.add_return_predictions(chars,settings, output_path)

In [60]:
print(chars.shape) #Skal være 73216 x 131
print(chars[['id', 'eom','pred_ld1']])

(73216, 131)
          id        eom  pred_ld1
0      10104 1986-12-31       NaN
1      10104 1987-01-31       NaN
2      10104 1987-02-28       NaN
3      10104 1987-03-31       NaN
4      10104 1987-04-30       NaN
...      ...        ...       ...
73211  92655 2022-07-31  0.010957
73212  92655 2022-08-31  0.009506
73213  92655 2022-09-30  0.010641
73214  92655 2022-10-31  0.010774
73215  92655 2022-11-30  0.011608

[73216 rows x 3 columns]


In [61]:
lambda_list = prepare_portfolio_data.create_lambda_list(chars)

In [62]:
print(lambda_list['2021-10-31'])

{10104: 2.1068747835833751e-10, 10107: 3.034783408033585e-11, 10138: 1.014425423141795e-09, 10696: 4.3268657868458776e-10, 11308: 2.5674127083269843e-10, 11850: 1.5210112818417197e-10, 11955: 9.846108938145744e-10, 12060: 2.392171855561806e-10, 12062: 1.1115228863141125e-09, 12490: 3.103196145302579e-10, 13688: 1.2451492409136745e-09, 13856: 3.0647710531490904e-10, 13901: 5.984566822586187e-10, 14008: 3.4654779374858333e-10, 14277: 5.572628200703795e-10, 14541: 1.8678925436727748e-10, 14593: 1.7957730786929064e-11, 14702: 1.9775531529792074e-10, 15720: 1.868244462399027e-09, 17778: 3.3642773554010653e-10, 18163: 1.9283146734748652e-10, 18411: 7.948401354253297e-10, 19393: 3.2583660018022594e-10, 19561: 8.240904270111589e-11, 20482: 3.1099118409527434e-10, 21371: 1.314659590664529e-09, 21936: 1.611785023727204e-10, 22103: 8.191041680175024e-10, 22111: 1.9039009050878924e-10, 22592: 4.542098255993858e-10, 22752: 2.035214983043305e-10, 25320: 1.6935230409360711e-09, 25785: 1.9152306625614

In [63]:
first_cov_date, hp_years, start_oos = prepare_portfolio_data.define_important_dates(barra_cov, settings)
date_ranges = prepare_portfolio_data.create_date_ranges(settings, first_cov_date, start_oos, hp_years)
dates_m1 = date_ranges["dates_m1"]
dates_m2 = date_ranges["dates_m2"]
dates_oos = date_ranges["dates_oos"]
dates_hp = date_ranges["dates_hp"]
#Disse er alle korrekte

# Base cases

In [64]:
print(wealth['eom'])
print(dates_oos)
print(pf_set)

0     1952-11-30
1     1952-12-31
2     1953-01-31
3     1953-02-28
4     1953-03-31
         ...    
837   2022-08-31
838   2022-09-30
839   2022-10-31
840   2022-11-30
841   2022-12-31
Name: eom, Length: 842, dtype: datetime64[ns]
DatetimeIndex(['2010-12-31', '2011-01-31', '2011-02-28', '2011-03-31',
               '2011-04-30', '2011-05-31', '2011-06-30', '2011-07-31',
               '2011-08-31', '2011-09-30',
               ...
               '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31',
               '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
               '2022-10-31', '2022-11-30'],
              dtype='datetime64[ns]', length=144, freq='ME')
{'wealth': 10000000000.0, 'gamma_rel': 10, 'mu': 0.007, 'lb_hor': 11}


In [65]:
ew = portfolio_choice_functions.ew_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set=pf_set)
mkt = portfolio_choice_functions.mkt_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set= pf_set)
rw = portfolio_choice_functions.rw_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set= pf_set)
mv = portfolio_choice_functions.mv_implement(data=chars, cov_list=barra_cov ,wealth=wealth, dates=dates_oos, pf_set= pf_set)
factor_ml = portfolio_choice_functions.factor_ml_implement(data = chars, wealth = wealth, dates= dates_oos, n_pfs=settings["factor_ml"]["n_pfs"]
, gam=pf_set['gamma_rel'])
tpf = portfolio_choice_functions.tpf_implement(data=chars, cov_list=barra_cov, wealth=wealth, dates=dates_oos, gam=pf_set["gamma_rel"])

# STATIC ML

In [ ]:
def create_lambda(x, ids):
    return np.diag([x[i] for i in ids])


In [96]:
import pandas as pd
import numpy as np
import itertools

# Eksempel: De brugerdefinerede funktioner skal defineres et andet sted
# def initial_weights_new(data, w_type): ...
# def create_cov(cov_item, ids): ...
# def sigma_gam_adj(sigma_gam, g, cov_type): ...
# def create_lambda(lambda_item, ids): ...
# def pf_ts_fun(data, **kwargs): ...

def static_val_fun(data, dates, cov_list, lambda_list, wealth, cov_type, gamma_rel, k=None, g=None, u=None, hps=None):
    """
    Beregner de statiske vægte for de angivne datoer.
    Parametre:
      - data: DataFrame med portefølje-data.
      - dates: Liste af datoer (som datetime-objekter).
      - cov_list: Dictionary med covariance-data pr. dato.
      - lambda_list: Dictionary med lambda-data pr. dato.
      - wealth: DataFrame med wealth-information (skal indeholde 'eom' og 'wealth', evt. 'mu_ld1').
      - cov_type: Type til justering af covariance.
      - gamma_rel: Skalar til vægtning.
      - k, g, u: Parametre, kan angives direkte.
      - hps: DataFrame med hyperparametre; hvis angivet, opdateres k, g og u for hver dato.
    """
    # Start med initiale weights (f.eks. VW, alt efter initial_weights_new)
    static_weights = General_Functions.initial_weights_new(data, w_type="vw")
    
    # Merge med kolonner fra data: id, eom, tr_ld1 og pred_ld1
    static_weights = static_weights.merge(data[['id', 'eom', 'tr_ld1', 'pred_ld1']], on=['id', 'eom'], how='left')
    # Merge med wealth for at få mu_ld1
    static_weights = static_weights.merge(wealth[['eom', 'mu_ld1']], on='eom', how='left')
    
    for d in dates:
        # Hvis hyperparametre (hps) er angivet, så opdater k, g og u
        if hps is not None:
            # Filtrer rækker hvor året for eom_ret er mindre end året for d
            hp_candidates = hps[hps['eom_ret'].dt.year < d.year]
            if not hp_candidates.empty:
                # Vælg den række med den seneste eom_ret
                hp = hp_candidates.loc[hp_candidates['eom_ret'].idxmax()]
                g = hp['g']
                u = hp['u']
                k = hp['k']
        
        # Hent wealth for datoen
        wealth_t_series = wealth.loc[wealth['eom'] == d, 'wealth']
        if wealth_t_series.empty:
            continue  # Eller håndtér fejlen, hvis der mangler data for d
        wealth_t = wealth_t_series.iloc[0]
        
        # Vælg de rækker i static_weights for datoen d
        mask = static_weights['eom'] == d
        # Sørg for, at id'erne er strenge (for at matche med de oprettede matricer)
        ids = static_weights.loc[mask, 'id'].tolist()

        
        # Byg covariance matrix og tilpas den
        sigma_gam = General_Functions.create_cov(cov_list[d], ids=ids) * gamma_rel

        sigma_gam = General_Functions.sigma_gam_adj(sigma_gam, g=g, cov_type=cov_type)
        
        # Byg lambda matrix og skaler med k
        lambda_val = create_lambda(lambda_list[d], ids=ids)
        lambda_val = lambda_val * k
        
        # Ekstraher de nødvendige kolonner som numpy-arrays
        pred_ld1 = static_weights.loc[mask, 'pred_ld1'].values
        w_start = static_weights.loc[mask, 'w_start'].values
        
        # Beregn højresidevektoren: pred_ld1 * u + wealth_t * (lambda @ w_start)
        rhs = (pred_ld1 * u) + wealth_t * (lambda_val.dot(w_start))
        # Matrix A: sigma_gam + wealth_t * lambda
        A = sigma_gam + wealth_t * lambda_val
        # Løs lineært system for at få nye vægte
        w_new = np.linalg.solve(A, rhs)
        # Tildel de beregnede vægte til kolonnen 'w' for de relevante rækker
        static_weights.loc[mask, 'w'] = w_new
        
        # Opdater startvægt for næste måned:
        try:
            current_index = list(dates).index(d)
            next_month = dates[current_index + 1]
        except IndexError:
            # Hvis der ikke er nogen næste dato, springes opdateringen over
            continue
        
        # Udregn 'w_opt_lag1' for de rækker, der svarer til d, men til næste måned
        temp = static_weights.loc[mask, ['id', 'w', 'tr_ld1', 'mu_ld1']].copy()
        temp['eom'] = next_month
        temp['w_opt_lag1'] = temp['w'] * (1 + temp['tr_ld1']) / (1 + temp['mu_ld1'])
        
        # Merge temp med static_weights på ['id', 'eom']
        static_weights = static_weights.merge(temp[['id', 'eom', 'w_opt_lag1']], on=['id', 'eom'], how='left')
        
        # Sæt w_start for rækker med eom == next_month
        next_mask = static_weights['eom'] == next_month
        static_weights.loc[next_mask, 'w_start'] = static_weights.loc[next_mask, 'w_opt_lag1']
        static_weights.loc[next_mask & static_weights['w_start'].isna(), 'w_start'] = 0
        
        # Fjern den midlertidige kolonne
        static_weights.drop(columns='w_opt_lag1', inplace=True)
        
    return static_weights


In [101]:
def static_implement(data_tc, cov_list, lambda_list, rf, wealth, mu, gamma_rel,
                     dates_full, dates_oos, dates_hp, hp_years,
                     k_vec, u_vec, g_vec, cov_type, validation=None, seed=None):
    """
    Fuldt statisk implementering.
    Parametre:
      - data_tc: DataFrame med tidsserie-data.
      - cov_list, lambda_list: Ordbøger med covariance- og lambda-data.
      - rf, wealth, mu, gamma_rel: Diverse parametre.
      - dates_full, dates_oos, dates_hp: Lister af datoer (skal være datetime-objekter).
      - hp_years: Årstal for HP-valg.
      - k_vec, u_vec, g_vec: Vektorer med hyperparametre.
      - cov_type: Typen af covariance.
      - validation: Valideringsresultater (kan være None, hvorefter de udregnes).
      - seed: Til eventuel reproducibilitet.
    """
    # Konverter nøglerne i cov_list og lambda_list til pd.Timestamp
    cov_list = {pd.to_datetime(key): value for key, value in cov_list.items()}
    lambda_list = {pd.to_datetime(key): value for key, value in lambda_list.items()}
    
    # Opret HP-grid: alle kombinationer af k, u og g
    static_hps = pd.DataFrame(list(itertools.product(k_vec, u_vec, g_vec)), columns=['k', 'u', 'g'])
    
    # Udvælg relevant data: valid == True og eom i dates_hp, med kolonnerne id, eom, me, tr_ld1, pred_ld1, sorteret efter id og eom
    data_rel = data_tc[(data_tc['valid'] == True) & (data_tc['eom'].isin(dates_hp))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
    data_rel = data_rel.sort_values(by=['id', 'eom'])
    
    # Hvis der ikke allerede er beregnet validation, udregnes den
    if validation is None:
        validation_list = []
        for i in range(len(static_hps)):
            print(i)
            hp = static_hps.iloc[i]
            static_w = static_val_fun(
                data_rel,
                dates=dates_hp,
                cov_list=cov_list,
                lambda_list=lambda_list,
                wealth=wealth,
                gamma_rel=gamma_rel,
                k=hp['k'],
                g=hp['g'],
                u=hp['u'],
                cov_type=cov_type
            )
            # Anvend pf_ts_fun på resultatet – denne funktion skal returnere en DataFrame med bl.a. 'eom_ret', 'r' og 'tc'
            static_w_pf = General_Functions.pf_ts_fun(static_w, data=data_tc, wealth=wealth, gam=gamma_rel)
            static_w_pf['hp_no'] = i
            static_w_pf['k'] = hp['k']
            static_w_pf['g'] = hp['g']
            static_w_pf['u'] = hp['u']
            validation_list.append(static_w_pf)
        validation = pd.concat(validation_list, ignore_index=True)
    
    # Sortér validation på hp_no og eom_ret
    validation.sort_values(by=['hp_no', 'eom_ret'], inplace=True)
    
    # Beregn kumulative middelværdier (cummean) per hp_no
    validation['cum_mean_r2'] = validation.groupby('hp_no')['r'].transform(lambda x: x.pow(2).expanding().mean())
    validation['cum_mean_r']  = validation.groupby('hp_no')['r'].transform(lambda x: x.expanding().mean())
    validation['cum_var'] = validation['cum_mean_r2'] - validation['cum_mean_r'] ** 2
    
    # Beregn cum_obj for hver hp_no: cummean(r - tc - 0.5 * cum_var * gamma_rel)
    def compute_cum_obj(group):
        return (group['r'] - group['tc'] - 0.5 * group['cum_var'] * gamma_rel).expanding().mean()
    
    validation['cum_obj'] = validation.groupby('hp_no', group_keys=False).apply(compute_cum_obj)
    
    # Beregn rangering inden for hver eom_ret-gruppe baseret på -cum_obj
    validation['rank'] = validation.groupby('eom_ret')['cum_obj'].rank(ascending=False, method='first')
    
    # Vælg de optimale HP'er: de rækker hvor eom_ret har måned == 12 og rank == 1, sorteret efter eom_ret
    optimal_hps = validation[(validation['eom_ret'].dt.month == 12) & (validation['rank'] == 1)].sort_values(by='eom_ret')
    
    # Implementér den endelige portefølje: vælg data_tc hvor eom i dates_oos og valid == True
    w_data = data_tc[(data_tc['eom'].isin(dates_oos)) & (data_tc['valid'] == True)][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
    w = static_val_fun(
        w_data,
        dates=dates_oos,
        cov_list=cov_list,
        lambda_list=lambda_list,
        wealth=wealth,
        gamma_rel=gamma_rel,
        hps=optimal_hps,
        cov_type=cov_type
    )
    
    pf = General_Functions.pf_ts_fun(w, data=data_tc, wealth=wealth, gam=gamma_rel)
    pf['type'] = "Static-ML*"
    
    return {"hps": validation, "best_hps": optimal_hps, "w": w, "pf": pf}


In [102]:
static = static_implement(
    data_tc=chars,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    rf=risk_free,              # Data
    wealth=wealth,
    mu=pf_set['mu'],
    gamma_rel=pf_set['gamma_rel'],  # Investor
    dates_full=dates_m1,
    dates_oos=dates_oos,
    dates_hp=dates_hp,
    hp_years=hp_years,          # Dates
    k_vec=settings['pf']['hps']['static']['k'],
    u_vec=settings['pf']['hps']['static']['u'],
    g_vec=settings['pf']['hps']['static']['g'],
    cov_type=settings['pf']['hps']['cov_type'],
    validation=None
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


C:\Users\andre\AppData\Local\Temp\ipykernel_4388\3287424247.py:67: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [103]:
static

{'hps':            inv  shorting  turnover         r        tc    eom_ret  hp_no    k  \
 0     0.569087  0.022263  0.483848 -0.006528  0.001461 2001-01-31      0  1.0   
 1     0.457976  0.054599  0.215799 -0.004249  0.000322 2001-02-28      0  1.0   
 2     0.447034  0.067469  0.170632 -0.005693  0.000200 2001-03-31      0  1.0   
 3     0.464901  0.082075  0.160113  0.021935  0.000160 2001-04-30      0  1.0   
 4     0.496974  0.107694  0.145025  0.002751  0.000140 2001-05-31      0  1.0   
 ...        ...       ...       ...       ...       ...        ...    ...  ...   
 7123  1.624196  0.447748  0.212894 -0.017031  0.000692 2022-08-31     26  0.2   
 7124  1.642989  0.469470  0.220085 -0.059857  0.000727 2022-09-30     26  0.2   
 7125  1.655115  0.478227  0.196233  0.071438  0.000702 2022-10-31     26  0.2   
 7126  1.686589  0.489138  0.188407  0.044224  0.000676 2022-11-30     26  0.2   
 7127  1.754330  0.530499  0.219457 -0.025422  0.000852 2022-12-31     26  0.2   
 
       

In [84]:
def create_lambda(x, ids):
    return np.diag([x[i] for i in ids])


In [97]:
# Konverter nøglerne i barra_cov til pd.Timestamp
barra_cov = {pd.to_datetime(key): value for key, value in barra_cov.items()}
lambda_list = {pd.to_datetime(key): value for key, value in lambda_list.items()}

# 1. Opret HP-grid: alle kombinationer af k, u og g
static_hps = pd.DataFrame(
    list(itertools.product(
        settings['pf']['hps']['static']['k'],
        settings['pf']['hps']['static']['u'],
        settings['pf']['hps']['static']['g']
    )),
    columns=['k', 'u', 'g']
)

# 2. Udvælg relevant data: valid == True og eom i dates_hp, med kolonnerne id, eom, me, tr_ld1, pred_ld1
data_rel = chars[(chars['valid'] == True) & (chars['eom'].isin(dates_hp))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
data_rel = data_rel.sort_values(by=['id', 'eom'])

# 3. Beregn validation, hvis den ikke allerede er defineret
if 'validation' not in globals() or validation is None:
    validation_list = []
    for i in range(len(static_hps)):
        print(i)
        hp = static_hps.iloc[i]
        static_w = static_val_fun(
            data_rel,
            dates=dates_hp,
            cov_list=barra_cov,
            lambda_list=lambda_list,
            wealth=wealth,
            gamma_rel=pf_set['gamma_rel'],
            k=hp['k'],
            g=hp['g'],
            u=hp['u'],
            cov_type=settings['pf']['hps']['cov_type']
        )
        # Anvend pf_ts_fun på resultatet – denne funktion skal returnere en DataFrame med bl.a. 'eom_ret', 'r' og 'tc'
        static_w_pf = General_Functions.pf_ts_fun(static_w, data=chars, wealth=wealth, gam=pf_set['gamma_rel'])
        static_w_pf['hp_no'] = i
        static_w_pf['k'] = hp['k']
        static_w_pf['g'] = hp['g']
        static_w_pf['u'] = hp['u']
        validation_list.append(static_w_pf)
    validation = pd.concat(validation_list, ignore_index=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [87]:
print("Nøgler i lambda_list for 2000-12-31:", lambda_list[pd.to_datetime('2000-12-31')].keys())


Nøgler i lambda_list for 2000-12-31: dict_keys([10104, 10107, 10138, 10696, 11308, 11850, 11955, 12060, 12062, 12490, 13688, 13856, 13901, 14008, 14277, 14541, 14593, 14702, 15720, 17778, 18163, 18411, 19393, 19561, 20482, 21371, 21936, 22103, 22111, 22592, 22752, 25320, 25785, 26403, 26710, 26825, 27828, 27887, 27959, 27983, 29647, 30681, 33099, 34746, 34833, 35554, 37584, 38703, 43449, 44644, 45356, 45751, 46886, 47896, 48506, 48725, 49154, 49656, 49680, 50876, 52038, 53613, 55976, 56274, 56573, 57665, 57904, 58246, 58683, 59010, 59176, 59328, 59408, 59459, 60097, 60442, 60628, 60871, 61399, 61621, 61735, 62148, 64186, 64282, 64311, 64390, 64936, 64995, 65875, 66093, 66157, 66181, 66800, 67774, 69032, 70033, 70519, 71298, 71563, 72726, 73139, 75034, 75186, 75510, 75607, 75825, 75828, 75844, 76076, 76226, 76605, 76712, 76841, 77178, 77274, 77462, 77605, 77606, 77702, 78045, 78405, 78975, 78987, 79057, 79145, 79237, 79323, 80070, 80924, 81055, 81061, 81132, 81284, 81472, 81540, 81696, 

In [100]:
# Konverter nøglerne i barra_cov til pd.Timestamp
barra_cov = {pd.to_datetime(key): value for key, value in barra_cov.items()}
lambda_list = {pd.to_datetime(key): value for key, value in lambda_list.items()}

# 1. Opret HP-grid: alle kombinationer af k, u og g
static_hps = pd.DataFrame(
    list(itertools.product(
        settings['pf']['hps']['static']['k'],
        settings['pf']['hps']['static']['u'],
        settings['pf']['hps']['static']['g']
    )),
    columns=['k', 'u', 'g']
)

# 2. Udvælg relevant data: valid == True og eom i dates_hp, med kolonnerne id, eom, me, tr_ld1, pred_ld1
data_rel = chars[(chars['valid'] == True) & (chars['eom'].isin(dates_hp))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
data_rel = data_rel.sort_values(by=['id', 'eom'])

# 3. Beregn validation, hvis den ikke allerede er defineret
if 'validation' not in globals() or validation is None:
    validation_list = []
    for i in range(len(static_hps)):
        print(i)
        hp = static_hps.iloc[i]
        static_w = static_val_fun(
            data_rel,
            dates=dates_hp,
            cov_list=barra_cov,
            lambda_list=lambda_list,
            wealth=wealth,
            gamma_rel=pf_set['gamma_rel'],
            k=hp['k'],
            g=hp['g'],
            u=hp['u'],
            cov_type=settings['pf']['hps']['cov_type']
        )
        # Anvend pf_ts_fun på resultatet – denne funktion skal returnere en DataFrame med bl.a. 'eom_ret', 'r' og 'tc'
        static_w_pf = General_Functions.pf_ts_fun(static_w, data=chars, wealth=wealth, gam=pf_set['gamma_rel'])
        static_w_pf['hp_no'] = i
        static_w_pf['k'] = hp['k']
        static_w_pf['g'] = hp['g']
        static_w_pf['u'] = hp['u']
        validation_list.append(static_w_pf)
    validation = pd.concat(validation_list, ignore_index=True)

# 5. Beregn kumulative middelværdier per hp_no:
#    Først: beregn kum. middel for r^2 og r
validation['cum_mean_r2'] = validation.groupby('hp_no')['r'].transform(lambda x: x.pow(2).expanding().mean())
validation['cum_mean_r']  = validation.groupby('hp_no')['r'].transform(lambda x: x.expanding().mean())
validation['cum_var'] = validation['cum_mean_r2'] - validation['cum_mean_r'] ** 2

# 6. Beregn cum_obj for hver hp_no: cummean(r - tc - 0.5 * cum_var * gamma_rel)
def compute_cum_obj(group):
    return (group['r'] - group['tc'] - 0.5 * group['cum_var'] * pf_set['gamma_rel']).expanding().mean()

validation['cum_obj'] = validation.groupby('hp_no', group_keys=False).apply(compute_cum_obj)

# 7. Beregn rangering inden for hver eom_ret-gruppe baseret på -cum_obj
validation['rank'] = validation.groupby('eom_ret')['cum_obj'].rank(ascending=False, method='first')

# 8. (Plottekoden er udeladt efter ønske.)

# 9. Vælg de optimale HP'er: de rækker hvor eom_ret har måned == 12 og rank == 1, sorteret efter eom_ret
optimal_hps = validation[(validation['eom_ret'].dt.month == 12) & (validation['rank'] == 1)].sort_values(by='eom_ret')

# 10. Implementér den endelige portefølje: vælg data_tc hvor eom i dates_oos og valid == True
w_data = chars[(chars['eom'].isin(dates_oos)) & (chars['valid'] == True)][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
w = static_val_fun(
    w_data,
    dates=dates_oos,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    wealth=wealth,
    gamma_rel=pf_set['gamma_rel'],
    hps=optimal_hps,
    cov_type=settings['pf']['hps']['cov_type']
)

pf = General_Functions.pf_ts_fun(w, data=chars, wealth=wealth, gam=pf_set['gamma_rel'])
pf['type'] = "Static-ML*"

# 11. Gem eller print resultaterne
result = {"hps": validation, "best_hps": optimal_hps, "w": w, "pf": pf}
print(result)

C:\Users\andre\AppData\Local\Temp\ipykernel_4388\3364318100.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


{'hps':            inv  shorting  turnover         r        tc    eom_ret  hp_no    k  \
0     0.569087  0.022263  0.483848 -0.006528  0.001461 2001-01-31      0  1.0   
1     0.457976  0.054599  0.215799 -0.004249  0.000322 2001-02-28      0  1.0   
2     0.447034  0.067469  0.170632 -0.005693  0.000200 2001-03-31      0  1.0   
3     0.464901  0.082075  0.160113  0.021935  0.000160 2001-04-30      0  1.0   
4     0.496974  0.107694  0.145025  0.002751  0.000140 2001-05-31      0  1.0   
...        ...       ...       ...       ...       ...        ...    ...  ...   
7123  1.624196  0.447748  0.212894 -0.017031  0.000692 2022-08-31     26  0.2   
7124  1.642989  0.469470  0.220085 -0.059857  0.000727 2022-09-30     26  0.2   
7125  1.655115  0.478227  0.196233  0.071438  0.000702 2022-10-31     26  0.2   
7126  1.686589  0.489138  0.188407  0.044224  0.000676 2022-11-30     26  0.2   
7127  1.754330  0.530499  0.219457 -0.025422  0.000852 2022-12-31     26  0.2   

        g     u  cu

In [111]:
result["pf"]["type"].unique()

array(['Static-ML*'], dtype=object)

In [106]:


# Sti til din base-mappe
base_path = "./data_fifty/"
# Hent liste over undermapper/filer


# Saml porteføljerne i én DataFrame inklusiv Static-ML
bm_pfs = pd.concat(
    [tpf["pf"], factor_ml["pf"], ew["pf"], mkt["pf"], rw["pf"], mv["pf"], static["pf"]], 
    ignore_index=True
)

# Gem resultatet som CSV
bm_pfs.to_csv(f"{output_path}/bms.csv", index=False)

# Læs CSV-filen med pandas
bm_pfs = pd.read_csv(os.path.join(base_path, "bms.csv"))

# Konverter eom_ret til dato
bm_pfs['eom_ret'] = pd.to_datetime(bm_pfs['eom_ret'])

# Erstat "Rank-Weighted" med "Rank-ML" i kolonnen type
bm_pfs.loc[bm_pfs['type'] == 'Rank-Weighted', 'type'] = 'Rank-ML'

pfs = bm_pfs


In [105]:
pfs = bm_pfs



array(['Markowitz-ML', 'Factor-ML', '1/N', 'Market', 'Rank-ML',
       'Minimum Variance', 'Static-ML*'], dtype=object)

In [107]:

gamma_rel = pf_set["gamma_rel"]  

# Beregn portefølje-sammenfatningsstatistikker
pf_summary = pfs.groupby("type").agg(
    n=("eom_ret", "count"),  # Antal observationer
    inv=("inv", "mean"),  # Gennemsnitlig investering
    shorting=("shorting", "mean"),  # Gennemsnitlig shorting
    turnover_notional=("turnover", "mean"),  # Gennemsnitlig omsætning (turnover)
    r=("r", lambda x: x.mean() * 12),  # Gennemsnitligt afkast annualiseret
    sd=("r", lambda x: x.std() * (12**0.5)),  # Volatilitet annualiseret
    sr_gross=("r", lambda x: x.mean() / x.std() * (12**0.5) if x.std() > 0 else None),  # Sharpe Ratio før transaktionsomkostninger
    tc=("tc", lambda x: x.mean() * 12),  # Gennemsnitlige transaktionsomkostninger annualiseret
    r_tc=("r", lambda x: (x - pfs.loc[x.index, "tc"]).mean() * 12),  # Nettoafkast efter transaktionsomkostninger
    sr=("r", lambda x: (x - pfs.loc[x.index, "tc"]).mean() / x.std() * (12**0.5) if x.std() > 0 else None),  # Sharpe Ratio efter transaktionsomkostninger
    obj=("r", lambda x: (x.mean() - 0.5 * x.var() * gamma_rel - pfs.loc[x.index, "tc"].mean()) * 12)  # Optimeringsmål
).reset_index()

# Sorter efter type
pf_summary = pf_summary.sort_values(by="type")

# Filtrer pfs til kun at inkludere de relevante typer (main_types skal defineres)
main_types = ["Markowitz-ML", "Factor-ML", "Market", "Rank-ML", "Minimum Variance", "Static-ML"]  # Tilpas efter behov
pfs = pfs[pfs["type"].isin(main_types)]
pfs["type"] = pd.Categorical(pfs["type"], categories=main_types, ordered=True)

# Vis resultatet
from IPython.display import display
display(pf_summary)

,type,n,inv,shorting,turnover_notional,r,sd,sr_gross,tc,r_tc,sr,obj
0,1/N,144,1.000000,0.000000,0.052444,0.129325,0.158812,0.814327,0.001649,0.127676,0.803943,0.001570
1,Factor-ML,144,2.000000,1.000000,1.823863,0.089088,0.161915,0.550212,2.586110,-2.497022,-15.421821,-2.628104
2,Market,144,1.000000,0.000000,0.008496,0.120273,0.143452,0.838420,0.000031,0.120242,0.838207,0.017350
3,Markowitz-ML,144,10.625595,4.999915,8.640049,0.209006,0.229274,0.911600,48.295589,-48.086583,-209.733899,-48.349416
4,Minimum Variance,144,2.346509,0.673255,1.181356,0.113696,0.117284,0.969407,0.620363,-0.506667,-4.320000,-0.575444
5,Rank-ML,144,2.000000,1.000000,0.987215,0.047434,0.086572,0.547911,0.451931,-0.404497,-4.672365,-0.441971
6,Static-ML*,144,0.799646,0.272905,0.058425,0.041964,0.039532,1.061521,0.000600,0.041364,1.046338,0.033550


In [108]:
# Antag, at du har en DataFrame 'pfs' og en liste med main_types:
main_types = ["Markowitz-ML", "Factor-ML", "Market", "Rank-ML", "Minimum Variance", "Static-ML"]

# Konverter 'eom_ret' til datetime
pfs['eom_ret'] = pd.to_datetime(pfs['eom_ret'])

# Udtræk unikke typer
types = pfs['type'].unique()

# Udregn kumulative summer pr. type
pfs['cumret'] = pfs.groupby('type', observed=False)['r'].cumsum()
pfs['cumret_tc'] = (pfs['r'] - pfs['tc']).groupby(pfs['type'], observed=False).cumsum()

# Filtrer 'pfs' for de ønskede typer og vælg de relevante kolonner
ts_data = pfs[pfs['type'].isin(main_types)][['type', 'eom_ret', 'cumret', 'cumret_tc']].copy()

# "Pivot longer" - konverter DataFrame'en fra bredt til langt format
ts_data = ts_data.melt(id_vars=['type', 'eom_ret'],
                       value_vars=['cumret', 'cumret_tc'],
                       var_name='name',
                       value_name='value')

# Opret ekstra rækker med 'eom_ret' sat til månedsafslutningen for den tidligste dato
min_date = pfs['eom_ret'].min()
eom = min_date + pd.offsets.MonthEnd(0)

extra_df = pd.DataFrame({
    'eom_ret': [eom] * (2 * len(types)),
    'type': np.tile(types, 2),
    'value': 0,
    'name': np.concatenate([
        np.repeat("cumret", len(types)),
        np.repeat("cumret_tc", len(types))
    ])
})

# Kombiner de oprindelige data med de ekstra rækker
ts_data = pd.concat([ts_data, extra_df], ignore_index=True)

# Tilføj en ny kolonne 'name_pretty' med pæne navne
mapping = {
    "cumret": "Gross return",
    "cumret_tc": "Return net of TC"
}
ts_data['name_pretty'] = ts_data['name'].map(mapping)
ts_data['name_pretty'] = pd.Categorical(ts_data['name_pretty'],
                                         categories=["Gross return", "Return net of TC"],
                                         ordered=True)
ts_data 

,type,eom_ret,name,value,name_pretty
0,Markowitz-ML,2011-01-31,cumret,-0.005601,Gross return
1,Markowitz-ML,2011-02-28,cumret,0.029230,Gross return
2,Markowitz-ML,2011-03-31,cumret,0.044245,Gross return
3,Markowitz-ML,2011-04-30,cumret,0.082471,Gross return
4,Markowitz-ML,2011-05-31,cumret,0.092591,Gross return
...,...,...,...,...,...
1445,Markowitz-ML,2011-01-31,cumret_tc,0.000000,Return net of TC
1446,Factor-ML,2011-01-31,cumret_tc,0.000000,Return net of TC
1447,Market,2011-01-31,cumret_tc,0.000000,Return net of TC
1448,Rank-ML,2011-01-31,cumret_tc,0.000000,Return net of TC
